In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType, ArrayType


In [ ]:
# Initialize Spark Session
spark = SparkSession.builder \
    .appName("AmazonReviewsSentimentAnalysis") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/29 11:56:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Define schema for Amazon reviews
review_schema = StructType([
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("overall", DoubleType(), True),
    StructField("summary", StringType(), True),
    StructField("unixReviewTime", LongType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("helpful", ArrayType(IntegerType()), True),
    StructField("category", StringType(), True)
])

# Load JSON data
raw_reviews = spark.read \
    .schema(review_schema) \
    .json("hdfs://namenode:9000/amazon_reviews/*.json")

print(f"Total records loaded: {raw_reviews.count()}")
raw_reviews.printSchema()

25/06/29 11:56:25 WARN FileSystem: Failed to initialize filesystem hdfs://namenode:9000/user/data/amazon_reviews/*.json.gz: java.lang.IllegalArgumentException: java.net.UnknownHostException: namenode
25/06/29 11:56:25 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: hdfs://namenode:9000/user/data/amazon_reviews/*.json.gz.
java.lang.IllegalArgumentException: java.net.UnknownHostException: namenode
	at org.apache.hadoop.security.SecurityUtil.buildTokenService(SecurityUtil.java:479)
	at org.apache.hadoop.hdfs.NameNodeProxiesClient.createProxyWithClientProtocol(NameNodeProxiesClient.java:134)
	at org.apache.hadoop.hdfs.DFSClient.<init>(DFSClient.java:370)
	at org.apache.hadoop.hdfs.DFSClient.<init>(DFSClient.java:309)
	at org.apache.hadoop.hdfs.DistributedFileSystem.initDFSClient(DistributedFileSystem.java:203)
	at org.apache.hadoop.hdfs.DistributedFileSystem.initialize(DistributedFileSystem.java:188)
	at org.apache.hadoop.fs.FileSy

IllegalArgumentException: java.net.UnknownHostException: namenode

In [ ]:
def clean_text(text):
    if text is None:
        return ""
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

clean_text_udf = udf(clean_text, StringType())